In [2]:
!pip3 install -q imblearn
!pip3 install -q boto3>=1.28.59

In [2]:
import pandas as pd 
import json

In [3]:
! ls ./data 

File-Analysis.xlsx
Predictions_v13_small_with_sec_numbering_cleaned.xlsx
Updated-Tagging.xlsx
agg_on_data_after_standard_tag_mapping.xlsx
all-data-dedup.json
all-data-raw.json
all-data.json
cleaned_tags_data.json
cleaned_tags_data_na_tag_lt3.json
cleaned_tags_data_na_tag_v2_lt3.json
dataset
input
output
raw_input
v9_train.xlsx


In [4]:
file_analysis_df = pd.read_excel("./data/File-Analysis.xlsx")

In [5]:
good_files = set(file_analysis_df.File.to_list()[:100])

In [11]:
df = pd.read_json("./data/all-data-raw.json")
df['tags'] = df['tags'].str.replace("Events of Default", "Event of Default")

In [12]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of unique files: {pd.unique(df.filename).shape[0]}")
print(f"Number of unique tags: {pd.unique(df.tags).shape[0]}")

Number of rows: 13032
Number of unique files: 135
Number of unique tags: 39


In [7]:
# df = df[df.filename.str.contains("Agreement", case=False)]

In [8]:
df = df.drop_duplicates('section_content', keep=False)

In [9]:
print(df.shape)

(11469, 7)


In [157]:
df = df[df.filename.map(lambda x: x in good_files)]

In [158]:
for t in df[['tags']].groupby('tags').size().reset_index().sort_values(by=0).itertuples(): 
    print(t.tags,":", t._2)

Compliance Certificate : 11
Mandatory Prepayments / Redemption : 14
Financial Statements : 23
Restricted Investments : 23
Optional Prepayment / Redemption : 26
Incremental Facilities : 40
Restricted Payments : 42
Permitted Indebtedness : 44
Consequences of Default : 45
Asset Disposition : 47
Transactions with Affiliates : 48
Prepayment : 56
Governing Laws : 61
Waterfall of Proceeds : 64
Event of Default : 72
Financial Covenant : 74
Additional Liens : 78
Premium and Fees : 78
Loan Repayment : 80
Reporting Requirements : 92
Interest Rate : 93
Facilities / Instrument : 135
NA : 6587


In [159]:
df.word_count.describe()

count    7833.000000
mean      518.638070
std       701.116055
min        21.000000
25%       127.000000
50%       276.000000
75%       611.000000
max      9381.000000
Name: word_count, dtype: float64

In [160]:
df.word_count.describe()

count    7833.000000
mean      518.638070
std       701.116055
min        21.000000
25%       127.000000
50%       276.000000
75%       611.000000
max      9381.000000
Name: word_count, dtype: float64

In [161]:
df[df.tags == "NA"][['tags', 'title', 'category']].groupby('tags').agg(set)

,title,category
tags,,
NA,"{, Payments Generally; Pro Rata Treatment; Sha...","{Negative Covenants, Financial Covenants, FYI,..."


In [162]:
agg_df = df[['tags', 'title' ]].groupby([ 'tags', 'title']).size().reset_index().sort_values(by=0).rename(columns={0:'freq'})

In [207]:
tag_title_filter = {t.tags+""+t.title for t in agg_df[agg_df.tags == "NA"][agg_df.freq <=3].itertuples()}

/tmp/ipykernel_18/497309628.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tag_title_filter = {t.tags+""+t.title for t in agg_df[agg_df.tags == "NA"][agg_df.freq <=3].itertuples()}


In [208]:
len(tag_title_filter)

1775

In [209]:
df['tag_title'] = df.tags+df.title

In [210]:
new_df = pd.concat([df[df.tags!="NA"], df[df.tag_title.isin( tag_title_filter)]])

In [211]:
new_df.word_count.describe()

count    3678.000000
mean      559.646003
std       742.116931
min        21.000000
25%       127.000000
50%       291.000000
75%       681.000000
max      9381.000000
Name: word_count, dtype: float64

In [212]:
new_df[['tags']].groupby('tags').size().reset_index()

,tags,0
0,Additional Liens,78
1,Asset Disposition,47
2,Compliance Certificate,11
3,Consequences of Default,45
4,Event of Default,72
5,Facilities / Instrument,135
6,Financial Covenant,74
7,Financial Statements,23
8,Governing Laws,61
9,Incremental Facilities,40


# Save Data cleaned for NA tags

In [213]:
new_df.to_json("./data/cleaned_tags_data_na_tag_lt3.json", orient='records')

# Create Train test split

In [214]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os

In [215]:
unique_filenames = new_df.filename.unique()

In [216]:
synth_df = pd.read_json("./data/output/synthesized_rows.json")
synth_df['tags'] = synth_df['tags'].str.replace("Events of Default", "Event of Default")

In [217]:

# Split filenames into train and test sets
train_files, test_files = train_test_split(unique_filenames, test_size=0.2, random_state=42)

# Create train and test datasets based on the split filenames
train_df = pd.concat([new_df[new_df['filename'].isin(train_files)].reset_index(), synth_df]).reset_index(drop=True)
test_df = new_df[new_df['filename'].isin(test_files)].reset_index()
del train_df['index'] 
del test_df['index'] 
train_df.shape, test_df.shape

((3085, 9), (764, 8))

In [218]:
selected_columns = ["section_content", "tags"]

In [219]:
train_tags = set(train_df.tags.unique())
test_tags = set(test_df.tags.unique())

In [220]:
print(test_tags - train_tags )
assert not test_tags - train_tags 

set()


In [221]:
OUTPUT_DIR = "./data/dataset/v7"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [222]:
train_df.head()

,title,category,tags,checklists,section_content,filename,word_count,tag_title,checklist
0,Commitments,Event Driven,Facilities / Instrument,[],Title is Commitments\nCategory is Event Driven...,"Paramount Global - $3,500,000,000 AMENDED AND ...",205,Facilities / InstrumentCommitments,NaN
1,Revolving Credit Loans; Competitive Loans,Event Driven,Facilities / Instrument,[],Title is Revolving Credit Loans; Competitive L...,"Paramount Global - $3,500,000,000 AMENDED AND ...",422,Facilities / InstrumentRevolving Credit Loans;...,NaN
2,Repayment of Loans,Event Driven,Loan Repayment,[],Title is Repayment of Loans\nCategory is Event...,"Paramount Global - $3,500,000,000 AMENDED AND ...",123,Loan RepaymentRepayment of Loans,NaN
3,Swingline Loans,Event Driven,Facilities / Instrument,[],Title is Swingline Loans\nCategory is Event Dr...,"Paramount Global - $3,500,000,000 AMENDED AND ...",1919,Facilities / InstrumentSwingline Loans,NaN
4,Letters of Credit,Event Driven,Facilities / Instrument,[],Title is Letters of Credit\nCategory is Event ...,"Paramount Global - $3,500,000,000 AMENDED AND ...",2735,Facilities / InstrumentLetters of Credit,NaN


In [223]:
train_df.to_json(os.path.join(OUTPUT_DIR, "train.json")) 
test_df.to_json(os.path.join(OUTPUT_DIR, "test.json"))

---

End of Training data creation with Undersampled tags data

--- 

# Create Undersampled dataset 

In [9]:
from imblearn.under_sampling import RandomUnderSampler

In [10]:
df[["filename", "section_content"]]

,filename,section_content
0,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",Title is Computation of Time Periods\nCategory...
1,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",Title is Accounting Terms\nCategory is FYI\nSe...
2,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",Title is Terms Generally\nCategory is FYI\nSec...
3,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",Title is Divisions\nCategory is FYI\nSection 1...
4,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",Title is The Advances\nCategory is FYI\nSectio...
...,...,...
13659,"Exelon Corp - $900,000,000 CREDIT AGREEMENT.se...",Title is Acknowledgement and Consent to Bail-I...
13660,"Exelon Corp - $900,000,000 CREDIT AGREEMENT.se...",Title is Acknowledgement Regarding Any Support...
13661,"Exelon Corp - $900,000,000 CREDIT AGREEMENT.se...",Title is Judgment Currency\nCategory is FYI\n9...
13662,"Exelon Corp - $900,000,000 CREDIT AGREEMENT.se...",Title is Payments Set Aside\nCategory is FYI\n...


In [11]:
df.values[0]

array(['Computation of Time Periods', 'FYI', 'NA', list([]),
       'Title is Computation of Time Periods\nCategory is FYI\nSection 1.02. Computation of Time Periods. In this Agreement, in the computation of periods of time from a specified date to a later specified date, the word “from” means “from and including”, the word “through” means “through and including” and each of the words “to” and “until” mean “to but excluding”.\n',
       'Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEMENT.section.json',
       51], dtype=object)

In [12]:
X = df[["filename", "section_content"]].values
# X = df['section_content'].values
y = df['tags'].values

In [13]:
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=42)
X_res, y_res = rus.fit_resample(df, y)

In [14]:
df['tags'] = df['tags'].str.replace("Events of Default", "Event of Default")

In [15]:
df[['tags']].groupby('tags').size().reset_index().sort_values(by=0)

,tags,0
7,Financial Statements,24
2,Compliance Certificate,25
12,Mandatory Prepayments / Redemption,30
19,Restricted Investments,33
14,Optional Prepayment / Redemption,47
9,Incremental Facilities,48
20,Restricted Payments,61
15,Permitted Indebtedness,63
21,Transactions with Affiliates,63
17,Prepayment,64


# Idenitfy Undersampled tags

In [16]:
label_counts = df[['tags']].groupby('tags').size().reset_index().rename(columns={0:"freq"}).sort_values(by='freq')

In [17]:
median_count = label_counts['freq'].median()

In [18]:
undersampled_labels = label_counts[label_counts.freq < median_count]

In [19]:
undersampled_labels

,tags,freq
7,Financial Statements,24
2,Compliance Certificate,25
12,Mandatory Prepayments / Redemption,30
19,Restricted Investments,33
14,Optional Prepayment / Redemption,47
9,Incremental Facilities,48
20,Restricted Payments,61
15,Permitted Indebtedness,63
21,Transactions with Affiliates,63
17,Prepayment,64


## Generate Content for Undersampled tags

In [20]:
import boto3 
import json 

In [21]:
bedrock = boto3.client(service_name="bedrock-runtime")

In [63]:
prompt = """You are a financial expert tasked with generating synthetic text from a given section of a Credit Agreement document. Here are the guidelines for generating synthetic content. 
- Maintain content length similar to the original content provided
- Modify the content slightly so that it can be used for training
- Do not deviate from the language of the original content
- You can modify the numbers, names of persons, companies, locations, or other nouns to generate the synthetic content. Maintain the consistency of the replaced names
- You can also replace the content with synonyms without overdoing it. 
Think deeply about the content and how it can be synthesized in the <thinking> tags.
Then finally, generate synthesized content in <content> tags. 
Here's the section content as follows: 
<section_content> 
{section_content}
</section_content> 
"""

In [73]:
def synthesize(section_content):
    content = prompt.format(section_content=section_content)
    body = json.dumps({
      "max_tokens": 4096,
      "messages": [{"role": "user", "content": content}],
      "anthropic_version": "bedrock-2023-05-31"
    })

    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")
    response_body = json.loads(response.get("body").read())
    return response_body.get("content")[0]['text']

In [ ]:
from tqdm import tqdm
import time
import re 

In [75]:
# synthesize  = lambda x: f"<content>{x}</content>"

In [76]:
synthesized_content = [] 

In [77]:
synth_size = len(synthesized_content)

In [78]:
def undersample_gen(): 
    for t in tqdm(undersampled_labels.itertuples()): 
        fill_count = int(median_count - t.freq )
        print(t.tags, fill_count)
        while True: 
            for row in tqdm(df[df.tags == t.tags].to_dict(orient='records')): 
                if fill_count == 0: 
                    break 
                yield row

                fill_count -= 1
            if fill_count == 0: 
                break 
    

In [79]:
samples = undersample_gen()

In [ ]:
for row in samples: 
    lines = row["section_content"].split("\n")
    result = re.findall("<content>((.|\n)+)</content>", synthesize(lines[2:]), flags=re.MULTILINE)
    content = "\n".join(lines[:2]) + result[0][0]
    synthesized_row = {
        'title': row['title'], 
        'category': row['category'], 
        'tags': row['tags'], 
        'checklist': row['checklists'], 
        'filename': "[synth]" + row['filename'], 
        'section_content': content, 
        'word_count': len(content.split())
    }
    synthesized_content.append(synthesized_row)


 67%|██████▋   | 20/30 [2:37:28<1:53:04, 678.46s/it]

In [94]:
pd.DataFrame(synthesized_content).to_json("./data/output/synthesized_rows.json")

In [93]:
pd.DataFrame(synthesized_content)[['tags']].groupby('tags').size().reset_index().rename(columns={0:"freq"}).sort_values(by='freq')

,tags,freq
2,Mandatory Prepayments / Redemption,44
0,Compliance Certificate,63
1,Financial Statements,64


In [91]:
print("Write finished!")

Write finished!


In [ ]:
# pd.DataFrame(synthesized_content).to_json("./data/output/synthesized_rows.json")

In [92]:
len(synthesized_content)

171

In [ ]:
section_content = "\n".join(df[df.tags == "Financial Statements"].head(10).to_dict(orient='records')[4]['section_content'].split("\n")[:2])
print(section_content)

In [ ]:
response = synthesize(section_content)
print(response)

In [ ]:
import re

In [ ]:
result = re.findall("<content>((.|\n)+)</content>", response, flags=re.MULTILINE)
print(result[0][0])

In [ ]:
X_res[['tags']].groupby("tags").size().rest_index()

In [ ]:
from collections import Counter

In [ ]:
Counter(y_res)

In [ ]:
from sklearn.model_selection import train_test_split

# Stratified split to maintain the balance of classes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)